In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 
showHeader('State Index')

In [ ]:
# fetch data set
df = fh.get_df(os.path.join(hndl.sandbox_dir,'workdf.parquet'))

In [ ]:
gb = df[(df.in_std_filtered)&(df.loc_within_state=='YES')].groupby(['bgStateName','bgCountyName','DisclosureId'],as_index=False)['date'].first()
gb1 = gb.groupby('bgStateName',as_index=False)['DisclosureId'].count().rename({'DisclosureId':'Number of disclosures'},axis=1)

gb2 = gb.groupby(['bgStateName','bgCountyName'],as_index=False)['DisclosureId'].count()
gb2 = gb2.groupby('bgStateName',as_index=False)['bgCountyName'].count().rename({'bgCountyName':'Number of counties'},axis=1)

gb3 = gb.groupby('bgStateName',as_index=False)['date'].agg(['min','max']).rename({'min':'earliest',
                                                                                  'max':'latest'},axis=1)
mg = pd.merge(gb1,gb2,on='bgStateName',how='left')
mg = pd.merge(mg,gb3,on='bgStateName',how='left')
mg = mg.rename({'bgStateName':'State'},axis=1)
# mg = mg.reset_index(drop=True)
mg.State = '<center><h3>'+mg.State.str.title().map(lambda x: th.getStateLink(x,x))+'</h3></center>'
mg = mg.sort_values('Number of disclosures',ascending=False)

import itables.options as opt
opt.order = []  # no sorting

iShow(mg.reset_index(drop=True),paging=False)